Import

In [12]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import detrend
import sys
import os, glob

Read

In [13]:
#_fc

# Define relative path to folder containing the text files

files_folder_fc = "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Data_fc"
files_fc = []

# Create a dataframe list by using a list comprehension

files_fc = [pd.read_csv(file, sep='\t') for file in glob.glob(os.path.join(files_folder_fc ,"*.txt"))]


df_fc = []
for i in files_fc:
    a=i.loc[:, ['Time[s]', 'COPx[cm]', 'COPy[cm]']].loc[(i['Time[s]']<= 35)& (i['Time[s]']>= 5)].rename(columns={"COPx[cm]": "COP_AP", "COPy[cm]": "COP_ML"})
    df_fc.append(a)



# function to get COM from COP 

def cogve(COP, freq, mass, height):
    """COGv estimation using COP data based on the inverted pendulum model.

    This function estimates the center of gravity vertical projection (COGv)
    displacement from the center of pressure (COP) displacement at the
    anterior-posterior direction during quiet upright standing. COP and COGv
    displacements are measurements useful to quantify the postural sway of a
    person while standing.

    The COGv displacement is estimated by low-pass filtering the COP
    displacement in the frequency domain according to the person's moment
    of rotational inertia as a single inverted pendulum [1]_.

    Parameters
    ----------
    COP    : 1D array_like
             center of pressure data [cm]
    freq   : float
             sampling frequency of the COP data
    mass   : float
             body mass of the subject [kg]
    height : float
             height of the subject [cm]

    Returns
    -------
    COGv   : 1D array
             center of gravity vertical projection data [cm]

    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/IP_Model.ipynb

    Examples
    --------
    >>> from cogve import cogve
    >>> y = np.cumsum(np.random.randn(3000))/50
    >>> cogv = cogve(y, freq=100, mass=70, height=170, show=True)
    """

    from scipy.signal._arraytools import odd_ext
    import scipy.fftpack

    COP = np.asarray(COP)
    height = height / 100  # cm to m
    g = 9.8  # gravity acceleration in m/s2
    # height of the COG w.r.t. ankle (McGinnis, 2005; Winter, 2005)
    hcog = 0.56 * height - 0.039 * height
    # body moment of inertia around the ankle
    # (Breniere, 1996), (0.0572 for the ml direction)
    I = mass * 0.0533 * height ** 2 + mass * hcog ** 2
    # Newton-Euler equation of motion for the inverted pendulum
    # COGv'' = w02*(COGv - COP)
    # where w02 is the squared pendulum natural frequency
    w02 = mass * g * hcog / I
    # add (pad) data and remove mean to avoid problems at the extremities
    COP = odd_ext(COP, n=freq)
    COPm = np.mean(COP)
    COP = COP - COPm
    # COGv is estimated by filtering the COP data in the frequency domain
    # using the transfer function for the inverted pendulum equation of motion
    N = COP.size
    COPfft = scipy.fftpack.fft(COP, n=N) / N  # COP fft
    w = 2 * np.pi * scipy.fftpack.fftfreq(n=N, d=1 / freq)  # angular frequency
    # transfer function
    TF = w02 / (w02 + w ** 2)
    COGv = np.real(scipy.fftpack.ifft(TF * COPfft) * N)
    COGv = COGv[0: N]
    # get back the mean and pad off data
    COP, COGv = COP + COPm, COGv + COPm
    COP, COGv = COP[freq: -freq], COGv[freq: -freq]

    return COGv


# Open patients info dataset
info_fc= pd.read_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Info/Fallers_Info_fc.csv')




# get COM from COP
df_com_fc=[]
for index in range(len(df_fc)):
    COM_AP = cogve(df_fc[index][['COP_AP', 'COP_ML']].values[:, 0]/100, freq=100, mass=float(info_fc.Weight.iloc[[index]]), height=float(info_fc.Height.iloc[[index]]))
    COM_ML = cogve(df_fc[index][['COP_AP', 'COP_ML']].values[:, 1]/100, freq=100, mass=float(info_fc.Weight.iloc[[index]]), height=float(info_fc.Height.iloc[[index]]))
    df_fc[index]['COM_AP']= list(COM_AP)
    df_fc[index]['COM_ML']= list(COM_ML)
    df_fc[index]= df_fc[index].loc[:, ['COM_AP', 'COM_ML']]
    df_com_fc.append(df_fc[index])



def save(lst_of_df_fc):
    for index in range(len(lst_of_df_fc)):
        lst_of_df_fc[index].to_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Processed_fc/FallersCOM_fc_[%s].csv' %index)


save(df_fc)
info_fc.to_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Info/Fallers_Info_withID_fc.csv')

In [14]:
#_fo

# Define relative path to folder containing the text files

files_folder_fo = "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Data_fo"
files_fo = []

# Create a dataframe list by using a list comprehension

files_fo = [pd.read_csv(file, sep='\t') for file in glob.glob(os.path.join(files_folder_fo ,"*.txt"))]


df_fo = []
for i in files_fo:
    a=i.loc[:, ['Time[s]', 'COPx[cm]', 'COPy[cm]']].loc[(i['Time[s]']<= 35)& (i['Time[s]']>= 5)].rename(columns={"COPx[cm]": "COP_AP", "COPy[cm]": "COP_ML"})
    df_fo.append(a)



# function to get COM from COP 

def cogve(COP, freq, mass, height):
    """COGv estimation using COP data based on the inverted pendulum model.

    This function estimates the center of gravity vertical projection (COGv)
    displacement from the center of pressure (COP) displacement at the
    anterior-posterior direction during quiet upright standing. COP and COGv
    displacements are measurements useful to quantify the postural sway of a
    person while standing.

    The COGv displacement is estimated by low-pass filtering the COP
    displacement in the frequency domain according to the person's moment
    of rotational inertia as a single inverted pendulum [1]_.

    Parameters
    ----------
    COP    : 1D array_like
             center of pressure data [cm]
    freq   : float
             sampling frequency of the COP data
    mass   : float
             body mass of the subject [kg]
    height : float
             height of the subject [cm]

    Returns
    -------
    COGv   : 1D array
             center of gravity vertical projection data [cm]

    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/IP_Model.ipynb

    Examples
    --------
    >>> from cogve import cogve
    >>> y = np.cumsum(np.random.randn(3000))/50
    >>> cogv = cogve(y, freq=100, mass=70, height=170, show=True)
    """

    from scipy.signal._arraytools import odd_ext
    import scipy.fftpack

    COP = np.asarray(COP)
    height = height / 100  # cm to m
    g = 9.8  # gravity acceleration in m/s2
    # height of the COG w.r.t. ankle (McGinnis, 2005; Winter, 2005)
    hcog = 0.56 * height - 0.039 * height
    # body moment of inertia around the ankle
    # (Breniere, 1996), (0.0572 for the ml direction)
    I = mass * 0.0533 * height ** 2 + mass * hcog ** 2
    # Newton-Euler equation of motion for the inverted pendulum
    # COGv'' = w02*(COGv - COP)
    # where w02 is the squared pendulum natural frequency
    w02 = mass * g * hcog / I
    # add (pad) data and remove mean to avoid problems at the extremities
    COP = odd_ext(COP, n=freq)
    COPm = np.mean(COP)
    COP = COP - COPm
    # COGv is estimated by filtering the COP data in the frequency domain
    # using the transfer function for the inverted pendulum equation of motion
    N = COP.size
    COPfft = scipy.fftpack.fft(COP, n=N) / N  # COP fft
    w = 2 * np.pi * scipy.fftpack.fftfreq(n=N, d=1 / freq)  # angular frequency
    # transfer function
    TF = w02 / (w02 + w ** 2)
    COGv = np.real(scipy.fftpack.ifft(TF * COPfft) * N)
    COGv = COGv[0: N]
    # get back the mean and pad off data
    COP, COGv = COP + COPm, COGv + COPm
    COP, COGv = COP[freq: -freq], COGv[freq: -freq]

    return COGv


# Open patients info dataset
info_fo= pd.read_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Info/Fallers_Info_fo.csv')




# get COM from COP
df_com_fo=[]
for index in range(len(df_fo)):
    COM_AP = cogve(df_fo[index][['COP_AP', 'COP_ML']].values[:, 0]/100, freq=100, mass=float(info_fo.Weight.iloc[[index]]), height=float(info_fo.Height.iloc[[index]]))
    COM_ML = cogve(df_fo[index][['COP_AP', 'COP_ML']].values[:, 1]/100, freq=100, mass=float(info_fo.Weight.iloc[[index]]), height=float(info_fo.Height.iloc[[index]]))
    df_fo[index]['COM_AP']= list(COM_AP)
    df_fo[index]['COM_ML']= list(COM_ML)
    df_fo[index]= df_fo[index].loc[:, ['COM_AP', 'COM_ML']]
    df_com_fo.append(df_fo[index])



def save(lst_of_df_fo):
    for index in range(len(lst_of_df_fo)):
        lst_of_df_fo[index].to_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Processed_fo/FallersCOM_fo_[%s].csv' %index)


save(df_fo)
info_fo.to_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Info/Fallers_Info_withID_fo.csv')


In [15]:
#_gc

# Define relative path to folder containing the text files

files_folder_gc = "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Data_gc"
files_gc = []

# Create a dataframe list by using a list comprehension

files_gc = [pd.read_csv(file, sep='\t') for file in glob.glob(os.path.join(files_folder_gc ,"*.txt"))]


df_gc = []
for i in files_gc:
    a=i.loc[:, ['Time[s]', 'COPx[cm]', 'COPy[cm]']].loc[(i['Time[s]']<= 35)& (i['Time[s]']>= 5)].rename(columns={"COPx[cm]": "COP_AP", "COPy[cm]": "COP_ML"})
    df_gc.append(a)



# function to get COM from COP 

def cogve(COP, freq, mass, height):
    """COGv estimation using COP data based on the inverted pendulum model.

    This function estimates the center of gravity vertical projection (COGv)
    displacement from the center of pressure (COP) displacement at the
    anterior-posterior direction during quiet upright standing. COP and COGv
    displacements are measurements useful to quantify the postural sway of a
    person while standing.

    The COGv displacement is estimated by low-pass filtering the COP
    displacement in the frequency domain according to the person's moment
    of rotational inertia as a single inverted pendulum [1]_.

    Parameters
    ----------
    COP    : 1D array_like
             center of pressure data [cm]
    freq   : float
             sampling frequency of the COP data
    mass   : float
             body mass of the subject [kg]
    height : float
             height of the subject [cm]

    Returns
    -------
    COGv   : 1D array
             center of gravity vertical projection data [cm]

    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/IP_Model.ipynb

    Examples
    --------
    >>> from cogve import cogve
    >>> y = np.cumsum(np.random.randn(3000))/50
    >>> cogv = cogve(y, freq=100, mass=70, height=170, show=True)
    """

    from scipy.signal._arraytools import odd_ext
    import scipy.fftpack

    COP = np.asarray(COP)
    height = height / 100  # cm to m
    g = 9.8  # gravity acceleration in m/s2
    # height of the COG w.r.t. ankle (McGinnis, 2005; Winter, 2005)
    hcog = 0.56 * height - 0.039 * height
    # body moment of inertia around the ankle
    # (Breniere, 1996), (0.0572 for the ml direction)
    I = mass * 0.0533 * height ** 2 + mass * hcog ** 2
    # Newton-Euler equation of motion for the inverted pendulum
    # COGv'' = w02*(COGv - COP)
    # where w02 is the squared pendulum natural frequency
    w02 = mass * g * hcog / I
    # add (pad) data and remove mean to avoid problems at the extremities
    COP = odd_ext(COP, n=freq)
    COPm = np.mean(COP)
    COP = COP - COPm
    # COGv is estimated by filtering the COP data in the frequency domain
    # using the transfer function for the inverted pendulum equation of motion
    N = COP.size
    COPfft = scipy.fftpack.fft(COP, n=N) / N  # COP fft
    w = 2 * np.pi * scipy.fftpack.fftfreq(n=N, d=1 / freq)  # angular frequency
    # transfer function
    TF = w02 / (w02 + w ** 2)
    COGv = np.real(scipy.fftpack.ifft(TF * COPfft) * N)
    COGv = COGv[0: N]
    # get back the mean and pad off data
    COP, COGv = COP + COPm, COGv + COPm
    COP, COGv = COP[freq: -freq], COGv[freq: -freq]

    return COGv


# Open patients info dataset
info_gc= pd.read_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Info/Fallers_Info_gc.csv')




# get COM from COP
df_com_gc=[]
for index in range(len(df_gc)):
    COM_AP = cogve(df_gc[index][['COP_AP', 'COP_ML']].values[:, 0]/100, freq=100, mass=float(info_gc.Weight.iloc[[index]]), height=float(info_gc.Height.iloc[[index]]))
    COM_ML = cogve(df_gc[index][['COP_AP', 'COP_ML']].values[:, 1]/100, freq=100, mass=float(info_gc.Weight.iloc[[index]]), height=float(info_gc.Height.iloc[[index]]))
    df_gc[index]['COM_AP']= list(COM_AP)
    df_gc[index]['COM_ML']= list(COM_ML)
    df_gc[index]= df_gc[index].loc[:, ['COM_AP', 'COM_ML']]
    df_com_gc.append(df_gc[index])



def save(lst_of_df_gc):
    for index in range(len(lst_of_df_gc)):
        lst_of_df_gc[index].to_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Processed_gc/FallersCOM_gc_[%s].csv' %index)


save(df_gc)
info_gc.to_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Info/Fallers_Info_withID_gc.csv')


In [16]:
#_go

# Define relative path to folder containing the text files

files_folder_go = "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Data_go"
files_go = []

# Create a dataframe list by using a list comprehension

files_go = [pd.read_csv(file, sep='\t') for file in glob.glob(os.path.join(files_folder_go ,"*.txt"))]


df_go = []
for i in files_go:
    a=i.loc[:, ['Time[s]', 'COPx[cm]', 'COPy[cm]']].loc[(i['Time[s]']<= 35)& (i['Time[s]']>= 5)].rename(columns={"COPx[cm]": "COP_AP", "COPy[cm]": "COP_ML"})
    df_go.append(a)



# function to get COM from COP 

def cogve(COP, freq, mass, height):
    """COGv estimation using COP data based on the inverted pendulum model.

    This function estimates the center of gravity vertical projection (COGv)
    displacement from the center of pressure (COP) displacement at the
    anterior-posterior direction during quiet upright standing. COP and COGv
    displacements are measurements useful to quantify the postural sway of a
    person while standing.

    The COGv displacement is estimated by low-pass filtering the COP
    displacement in the frequency domain according to the person's moment
    of rotational inertia as a single inverted pendulum [1]_.

    Parameters
    ----------
    COP    : 1D array_like
             center of pressure data [cm]
    freq   : float
             sampling frequency of the COP data
    mass   : float
             body mass of the subject [kg]
    height : float
             height of the subject [cm]

    Returns
    -------
    COGv   : 1D array
             center of gravity vertical projection data [cm]

    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/IP_Model.ipynb

    Examples
    --------
    >>> from cogve import cogve
    >>> y = np.cumsum(np.random.randn(3000))/50
    >>> cogv = cogve(y, freq=100, mass=70, height=170, show=True)
    """

    from scipy.signal._arraytools import odd_ext
    import scipy.fftpack

    COP = np.asarray(COP)
    height = height / 100  # cm to m
    g = 9.8  # gravity acceleration in m/s2
    # height of the COG w.r.t. ankle (McGinnis, 2005; Winter, 2005)
    hcog = 0.56 * height - 0.039 * height
    # body moment of inertia around the ankle
    # (Breniere, 1996), (0.0572 for the ml direction)
    I = mass * 0.0533 * height ** 2 + mass * hcog ** 2
    # Newton-Euler equation of motion for the inverted pendulum
    # COGv'' = w02*(COGv - COP)
    # where w02 is the squared pendulum natural frequency
    w02 = mass * g * hcog / I
    # add (pad) data and remove mean to avoid problems at the extremities
    COP = odd_ext(COP, n=freq)
    COPm = np.mean(COP)
    COP = COP - COPm
    # COGv is estimated by filtering the COP data in the frequency domain
    # using the transfer function for the inverted pendulum equation of motion
    N = COP.size
    COPfft = scipy.fftpack.fft(COP, n=N) / N  # COP fft
    w = 2 * np.pi * scipy.fftpack.fftfreq(n=N, d=1 / freq)  # angular frequency
    # transfer function
    TF = w02 / (w02 + w ** 2)
    COGv = np.real(scipy.fftpack.ifft(TF * COPfft) * N)
    COGv = COGv[0: N]
    # get back the mean and pad off data
    COP, COGv = COP + COPm, COGv + COPm
    COP, COGv = COP[freq: -freq], COGv[freq: -freq]

    return COGv


# Open patients info dataset
info_go= pd.read_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Info/Fallers_Info_go.csv')




# get COM from COP
df_com_go=[]
for index in range(len(df_go)):
    COM_AP = cogve(df_go[index][['COP_AP', 'COP_ML']].values[:, 0]/100, freq=100, mass=float(info_go.Weight.iloc[[index]]), height=float(info_go.Height.iloc[[index]]))
    COM_ML = cogve(df_go[index][['COP_AP', 'COP_ML']].values[:, 1]/100, freq=100, mass=float(info_go.Weight.iloc[[index]]), height=float(info_go.Height.iloc[[index]]))
    df_go[index]['COM_AP']= list(COM_AP)
    df_go[index]['COM_ML']= list(COM_ML)
    df_go[index]= df_go[index].loc[:, ['COM_AP', 'COM_ML']]
    df_com_go.append(df_go[index])



def save(lst_of_df_go):
    for index in range(len(lst_of_df_go)):
        lst_of_df_go[index].to_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Processed_go/FallersCOM_go_[%s].csv' %index)


save(df_go)
info_go.to_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Info/Fallers_Info_withID_go.csv')
